## Формирование обучающей и тестовой выборок (rating-based)

В этом ноутбуке мы формируем датасеты на основе взаимодействий пользователей с товарами, используя оценку `rating` как основу для бинарной разметки:

- `label = 1`, если `rating >= 4`, иначе `0`
- Обучающая и тестовая выборки формируются **по времени**: `2015–2017` — train, `2018` — test

- `df_train_baseline_0_rating_based.csv` — сбалансированная выборка позитивных и негативных примеров для baseline-0 модели, сгенерированная подходом rating-based;
- `df_train_baseline_1_rating_based.csv` — сбалансированная выборка позитивных и негативных примеров для baseline-1 модели, сгенерированная подходом rating-based;
- `df_train_CLIP_rating_based.csv` — сбалансированная выборка позитивных и негативных примеров для мэтчинговой модел, сгенерированная подходом rating-based;
- `df_test_ground_truth_rating_based.csv` — тестовый лог взаимодействий за выбранный временной период, где положительным взаимодействием будет считаться факт выставления оценки >= 4.

Также добавляется словарь `ground_truth`, необходимый для расчёта метрик `Precision@K`, `MAP@K`.


In [1]:
# Импорт требуемых библиотек
import pandas as pd
import datetime
import numpy as np

In [2]:
# Подгрузим датасет с отзывами
df = pd.read_csv("data/amazon_interactions_filtered.csv")

In [3]:
# Ограничим временной диапазон: только 2015–2018
df = df[df["timestamp"] >= pd.Timestamp("2015-01-01").timestamp()].copy()

# Сортировка по времени
df = df.sort_values("timestamp")

# Формируем train/test сплит:
df_train = df[df["timestamp"] < pd.Timestamp("2018-01-01").timestamp()]
df_test = df[(df["timestamp"] >= pd.Timestamp("2018-01-01").timestamp()) &
             (df["timestamp"] < pd.Timestamp("2018-10-01").timestamp())]

## Построение меток (rating-based label)

Вместо генерации негативных примеров (1:1 sampling) мы используем оценки пользователей:

- `label = 1`, если `rating >= 4` (положительная оценка)
- `label = 0`, если `rating < 4` (нейтральная/негативная)

Такой подход позволяет напрямую использовать силу пользовательской оценки.


In [4]:
# Формируем бинарную метку: label = 1, если рейтинг >= 4
df_train["label"] = (df_train["rating"] >= 4).astype(int)
df_test["label"] = (df_test["rating"] >= 4).astype(int)

C:\Users\Администратор\AppData\Local\Temp\ipykernel_2924\1089353448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["label"] = (df_train["rating"] >= 4).astype(int)
C:\Users\Администратор\AppData\Local\Temp\ipykernel_2924\1089353448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["label"] = (df_test["rating"] >= 4).astype(int)


In [5]:
# Убираем рейтинг, чтобы он не утёк в обучение
df_train_final = df_train[["user_id", "item_id", "label"]].copy()
df_test_final = df_test[["user_id", "item_id", "label"]].copy()

In [6]:
print(f"Train: {len(df_train):,} строк, {df_train['user_id'].nunique():,} пользователей, {df_train['item_id'].nunique():,} товаров")
print(f"Test:  {len(df_test):,} строк, {df_test['user_id'].nunique():,} пользователей, {df_test['item_id'].nunique():,} товаров")

Train: 1,873,171 строк, 646,362 пользователей, 96,323 товаров
Test:  311,657 строк, 165,753 пользователей, 49,234 товаров


In [7]:
# Выделим уникальных юзеров на трейне и тесте
users_train = set(df_train["user_id"])
users_test = set(df_test["user_id"])

# Сколько юзеров в test уже встречались в train
overlap_users = users_test & users_train

In [8]:
print(f"Всего пользователей в test: {len(users_test):,}")
print(f"Из них уже были в train: {len(overlap_users):,}")
print(f"Доля покрытых пользователей: {round(len(overlap_users) / len(users_test) * 100, 2)}%")

Всего пользователей в test: 165,753
Из них уже были в train: 101,967
Доля покрытых пользователей: 61.52%


Модель не обучается для cold-start пользователей, поэтому пользователи без истории во временном трейне исключены из теста.

In [9]:
# Оставим в тестовой выборке только тех юзеров, кто есть в train
df_test = df_test[df_test["user_id"].isin(users_train)].copy()
print(f"Тест после фильтрации: {len(df_test):,} строк, {df_test['user_id'].nunique():,} пользователей")

Тест после фильтрации: 169,898 строк, 101,967 пользователей


In [10]:
# Подгрузим датасет со сгенерированными фичами товаров
df_meta = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)

In [11]:
print("Проверим пропуски после загрузки:")
print(df_meta[["brand", "category_main", "price_clean"]].isna().sum())

Проверим пропуски после загрузки:
brand            0
category_main    0
price_clean      0
dtype: int64


In [12]:
# Удаляем CLIP-фичи
clip_cols = [col for col in df_meta.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
df_meta = df_meta.drop(columns=clip_cols)

In [13]:
print(df_meta.shape)
df_meta.head(2)

(148948, 112)


,asin,title_len,title_has_digit,description_text_len,text_full,brand,is_top20_brand,price_clean,has_price,category_main,...,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99,tfidf_100
0,0853347867,-1.251232,0,-0.631098,"Trim Healthy Mama Xylitol. Shipped from UK, pl...",Unknown,1,-0.518064,0,Cooking & Baking,...,1.099800,0.677096,1.538071,-0.064891,-0.611596,2.097358,-1.535881,-0.427645,1.433121,0.793126
1,4639725043,-0.735001,1,-0.426547,Lipton Yellow Label Tea (loose tea) - 450g. Li...,Lipton,0,0.061373,1,Beverages,...,-0.016708,0.899488,-0.328298,1.685778,0.418503,1.372579,-0.176657,-0.481735,-1.121662,0.877780


In [14]:
# Проверим, что названия колонок совпадают
print("asin" in df_meta.columns)
print("item_id" in df_train_final.columns)

True
True


In [15]:
# Переименуем asin в item_id
df_meta.rename(columns={"asin": "item_id"}, inplace=True)

### Для baseline 0

In [16]:
# Мерджим трейн с фичами товаров
df_train_merged = df_train_final.merge(
    df_meta,
    on="item_id",
    how="left" # сохраняем все примеры, даже если нет мета-данных
)

In [17]:
print(df_train_merged.shape)
print(df_train_merged["label"].value_counts())
print("Доля строк с отсутствующей мета-информацией:", df_train_merged.isna().mean().round(3))

(1873171, 114)
label
1    1573593
0     299578
Name: count, dtype: int64
Доля строк с отсутствующей мета-информацией: user_id            0.000
item_id            0.000
label              0.000
title_len          0.127
title_has_digit    0.127
                   ...  
tfidf_96           0.127
tfidf_97           0.127
tfidf_98           0.127
tfidf_99           0.127
tfidf_100          0.127
Length: 114, dtype: float64


In [18]:
# Отфильтруем данные с пропусками в нижеупомянутых полях
required_fields = ["text_full", "image_main", "brand"]

df_train_clean = df_train_merged.dropna(subset=required_fields).copy()
print(f"После удаления строк без критичных признаков: {len(df_train_clean):,} строк")

После удаления строк без критичных признаков: 1,635,089 строк


In [19]:
# Создадим ground truth - словарь, где:
# ключ — user_id
# значение — множество товаров, с которыми он взаимодействовал в test
# с label = 1
df_test = df_test[df_test["label"] == 1]
ground_truth = df_test.groupby("user_id")["item_id"].apply(set).to_dict()

### Ground truth и оценка по рейтингу

В этом подходе мы считаем, что пользователь взаимодействовал с товаром, **если он поставил ему рейтинг >= 4**.

Мы используем `test`-датасет и считаем `ground_truth` как множество товаров с положительной оценкой (`label = 1`) для каждого пользователя.

#### Что мы делаем:

- Отбираем `item_id`, у которых `label = 1` в тесте.
- Группируем их по `user_id`, чтобы получить `ground_truth[user] - set(товаров)`.
- Это будет использоваться для подсчёта метрик `Precision@k`, `MAP@k`.


In [20]:
df_train_clean.head(1)

,user_id,item_id,label,title_len,title_has_digit,description_text_len,text_full,brand,is_top20_brand,price_clean,...,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99,tfidf_100
0,A291K1IIC03N7W,B004UETCWO,0,-0.370603,1.0,0.019311,"Robinson's Orange Barley Water, 28.7-Ounce (Pa...",Robinsons,0.0,0.557295,...,-1.275664,-0.911563,1.150737,-3.58037,-0.024295,4.119833,-0.051605,1.528907,-3.408201,0.866693


In [21]:
# Сохраняем данные

# Тренировочная выборка с мета-признаками
df_train_clean.to_csv("data/df_train_baseline_0_rating_based.csv", index=False)

# Тест как лог взаимодействий для label по рейтингу
# ground_truth можно будет пересчитать по df_test в следующем ноутбуке
df_test.to_csv("data/df_test_ground_truth_rating_based.csv", index=False)

### baseline 1

In [22]:
# Загружаем мета-датасет с уже готовыми tfidf_фичами
df_meta_tfidf = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],
    keep_default_na=False
)

In [23]:
# Переименуем asin в item_id
df_meta_tfidf.rename(columns={"asin": "item_id"}, inplace=True)

In [24]:
# Удалим clip-эмбеддинги (они здесь не нужны)
clip_cols = [col for col in df_meta_tfidf.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
df_meta_tfidf = df_meta_tfidf.drop(columns=clip_cols)

In [25]:
# Мерджим трейн с фичами товаров после tf-idf
# для baseline 1
df_train_merged = df_train_final.merge(
    df_meta_tfidf,
    on="item_id",
    how="left"
)

In [26]:
print(df_train_merged.shape)
print(df_train_merged["label"].value_counts())
print("Доля строк с отсутствующей мета-информацией:", df_train_merged.isna().mean().round(3))

(1873171, 114)
label
1    1573593
0     299578
Name: count, dtype: int64
Доля строк с отсутствующей мета-информацией: user_id            0.000
item_id            0.000
label              0.000
title_len          0.127
title_has_digit    0.127
                   ...  
tfidf_96           0.127
tfidf_97           0.127
tfidf_98           0.127
tfidf_99           0.127
tfidf_100          0.127
Length: 114, dtype: float64


In [27]:
# Отфильтруем данные с пропусками в нижеупомянутых полях
required_fields = ["text_full", "image_main", "brand"]

df_train_clean = df_train_merged.dropna(subset=required_fields).copy()
print(f"После удаления строк без критичных признаков: {len(df_train_clean):,} строк")

После удаления строк без критичных признаков: 1,635,089 строк


In [28]:
# Удалим уже не нужные поля text_full и image_main
df_train_clean.drop(columns=["text_full", "image_main"], inplace=True)

In [29]:
# Выведем итоговый список полей
df_train_clean.columns.tolist()

['user_id',
 'item_id',
 'label',
 'title_len',
 'title_has_digit',
 'description_text_len',
 'brand',
 'is_top20_brand',
 'price_clean',
 'has_price',
 'category_main',
 'is_top9_category_main',
 'tfidf_1',
 'tfidf_2',
 'tfidf_3',
 'tfidf_4',
 'tfidf_5',
 'tfidf_6',
 'tfidf_7',
 'tfidf_8',
 'tfidf_9',
 'tfidf_10',
 'tfidf_11',
 'tfidf_12',
 'tfidf_13',
 'tfidf_14',
 'tfidf_15',
 'tfidf_16',
 'tfidf_17',
 'tfidf_18',
 'tfidf_19',
 'tfidf_20',
 'tfidf_21',
 'tfidf_22',
 'tfidf_23',
 'tfidf_24',
 'tfidf_25',
 'tfidf_26',
 'tfidf_27',
 'tfidf_28',
 'tfidf_29',
 'tfidf_30',
 'tfidf_31',
 'tfidf_32',
 'tfidf_33',
 'tfidf_34',
 'tfidf_35',
 'tfidf_36',
 'tfidf_37',
 'tfidf_38',
 'tfidf_39',
 'tfidf_40',
 'tfidf_41',
 'tfidf_42',
 'tfidf_43',
 'tfidf_44',
 'tfidf_45',
 'tfidf_46',
 'tfidf_47',
 'tfidf_48',
 'tfidf_49',
 'tfidf_50',
 'tfidf_51',
 'tfidf_52',
 'tfidf_53',
 'tfidf_54',
 'tfidf_55',
 'tfidf_56',
 'tfidf_57',
 'tfidf_58',
 'tfidf_59',
 'tfidf_60',
 'tfidf_61',
 'tfidf_62',
 'tfidf

In [30]:
# Вторая тренировочная выборка для baseline 1
# с признаками товаров после tf-idf
df_train_clean.to_csv("data/df_train_baseline_1_rating_based.csv", index=False)

CLIP-эмбеддинги (готовые)

In [31]:
# Подгружаем эмбеддинги и мета-фичи
df_meta_clip = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],
    keep_default_na=False
)

In [32]:
# Переименуем asin в item_id
df_meta_clip.rename(columns={"asin": "item_id"}, inplace=True)

In [33]:
# Удалим tfidf-фичи (они здесь не нужны)
tfidf_cols = [col for col in df_meta_clip.columns if col.startswith("tfidf_")]
df_meta_clip = df_meta_clip.drop(columns=tfidf_cols)

In [34]:
# Мерджим трейн с фичами товаров после CLIP-энкодера
# для мэтчинговой модели
df_train_merged_clip = df_train_final.merge(
    df_meta_clip,
    on="item_id",
    how="left"
)

In [35]:
# Отфильтруем данные с пропусками в нижеупомянутых полях
required_fields = ["text_full", "image_main", "brand"]
df_train_clip_clean = df_train_merged_clip.dropna(subset=required_fields).copy()
print(f"После удаления строк без критичных признаков: {len(df_train_clip_clean):,} строк")

После удаления строк без критичных признаков: 1,635,089 строк


In [36]:
# Удалим текст и картинки (они уже в эмбеддингах)
df_train_clip_clean.drop(columns=["text_full", "image_main"], inplace=True)

In [37]:
# Сохраняем CLIP-трейн
df_train_clip_clean.to_csv("data/df_train_CLIP_rating_based.csv", index=False)

### Выводы и сохранение данных

- Итоговые обучающие выборки (`df_train_baseline_0_rating_based.csv`, `df_train_baseline_1_rating_based.csv`, `df_train_CLIP_rating_based.csv`) содержат сбалансированные пары `(user, item)` с метками 0 и 1 (проставленных на основе rating-based подхода).
- Тестовая выборка (`df_test_ground_truth_rating_based.csv`) представляет собой лог реальных взаимодействий за выбранный год, на основе которого будет построен `ground_truth` — словарь соответствия `user_id → set(item_id)`, используемый для оценки рекомендаций.

Эти данные будут использоваться в ноутбуке с обучением и валидацией моделей.